In [ ]:
pip install -r requirements.txt

In [7]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

hf_token = config["huggingface"]["token"]
login("hf_token")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [15]:
!kaggle datasets download -d sbhatti/news-summarization

Dataset URL: https://www.kaggle.com/datasets/sbhatti/news-summarization
License(s): CC0-1.0
100%|█████████████████████████████████████▉| 1.37G/1.37G [00:44<00:00, 40.7MB/s]
100%|██████████████████████████████████████| 1.37G/1.37G [00:44<00:00, 33.2MB/s]


In [16]:
import zipfile
with zipfile.ZipFile('news-summarization.zip', 'r') as zip_ref:
    zip_ref.extractall('news-summarization')